In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
!head data/72565003017.csv

"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","AA1","AA2","AA3","AB1","AD1","AE1","AH1","AH2","AH3","AH4","AH5","AH6","AI1","AI2","AI3","AI4","AI5","AI6","AJ1","AK1","AM1","AN1","AT1","AT2","AT3","AT4","AT5","AT6","AU1","AU2","AU3","AW1","AW2","AW3","AW4","AX1","AX2","AX3","AX4","ED1","GA1","GA2","GA3","GD1","GD2","GD3","GD4","GE1","GF1","KA1","KA2","KB1","KB2","KB3","KC1","KC2","KD1","KD2","KE1","KG1","KG2","MA1","MD1","MF1","MG1","MH1","MK1","MV1","MW1","MW2","OC1","OD1","OE1","OE2","OE3","RH1","RH2","RH3","WA1","REM","EQD"
"72565003017","2021-01-01T00:00:00","4","39.84657","-104.65623","1647.2","DENVER INTERNATIONAL AIRPORT, CO US","FM-12","99999","V020","340,1,N,0021,1","99999,9,9,N","016000,1,9,9","+0000,1","-0056,1","10162,1",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"99,9,+02250,1,99,9",,,,,,,"9,MSL   ,+99999,+99999","06,99,1,99,9,99,9,02250,1,99,9,99,9","120,M,+0061,1","120,N,-00

In [22]:
denver = pd.read_csv(
    'data/72565003017.csv',
    parse_dates=['DATE'],
    dtype={'AT5':object, 'AT6':object, 'AX4':object, 'WA1':object})

In [23]:
denver.head()

,STATION,DATE,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,CALL_SIGN,QUALITY_CONTROL,...,OD1,OE1,OE2,OE3,RH1,RH2,RH3,WA1,REM,EQD
0,72565003017,2021-01-01 00:00:00,4,39.84657,-104.65623,1647.2,"DENVER INTERNATIONAL AIRPORT, CO US",FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN08072565 32866 63404 10000 21056 38317 4016...,NaN
1,72565003017,2021-01-01 00:53:00,7,39.84657,-104.65623,1647.2,"DENVER INTERNATIONAL AIRPORT, CO US",FM-15,KDEN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10712/31/20 17:53:03 METAR KDEN 010053Z 340...,NaN
2,72565003017,2021-01-01 01:53:00,7,39.84657,-104.65623,1647.2,"DENVER INTERNATIONAL AIRPORT, CO US",FM-15,KDEN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10812/31/20 18:53:03 METAR KDEN 010153Z 340...,NaN
3,72565003017,2021-01-01 02:53:00,7,39.84657,-104.65623,1647.2,"DENVER INTERNATIONAL AIRPORT, CO US",FM-15,KDEN,V030,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MET10612/31/20 19:53:03 METAR KDEN 010253Z 000...,NaN
4,72565003017,2021-01-01 03:00:00,4,39.84657,-104.65623,1647.2,"DENVER INTERNATIONAL AIRPORT, CO US",FM-12,99999,V020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SYN06472565 32966 60000 10000 21078 38343 4018...,NaN


In [24]:
denver.shape

(13426, 97)

In [25]:
denver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13426 entries, 0 to 13425
Data columns (total 97 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   STATION          13426 non-null  int64         
 1   DATE             13426 non-null  datetime64[ns]
 2   SOURCE           13426 non-null  int64         
 3   LATITUDE         13426 non-null  float64       
 4   LONGITUDE        13426 non-null  float64       
 5   ELEVATION        13426 non-null  float64       
 6   NAME             13426 non-null  object        
 7   REPORT_TYPE      13426 non-null  object        
 8   CALL_SIGN        13426 non-null  object        
 9   QUALITY_CONTROL  13426 non-null  object        
 10  WND              13426 non-null  object        
 11  CIG              13426 non-null  object        
 12  VIS              13426 non-null  object        
 13  TMP              13426 non-null  object        
 14  DEW              13426 non-null  objec

In [26]:
denver.describe()

,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION
count,1.342600e+04,13426.000000,1.342600e+04,1.342600e+04,1.342600e+04
mean,7.256500e+10,6.302622,3.984657e+01,-1.046562e+02,1.647200e+03
std,0.000000e+00,1.221638,7.105692e-15,4.263415e-14,4.547643e-13
min,7.256500e+10,4.000000,3.984657e+01,-1.046562e+02,1.647200e+03
25%,7.256500e+10,6.000000,3.984657e+01,-1.046562e+02,1.647200e+03
50%,7.256500e+10,7.000000,3.984657e+01,-1.046562e+02,1.647200e+03
75%,7.256500e+10,7.000000,3.984657e+01,-1.046562e+02,1.647200e+03
max,7.256500e+10,7.000000,3.984657e+01,-1.046562e+02,1.647200e+03


In [ ]:
j